In [19]:
import datetime
import pandas as pd
import os

library_books = [] 
borrowed_books =[]
logs = [] 
admin_username = 'admin'
admin_pass = '12&qw'

def display_main_menu():
    print("Library Management System")
    print("1. Add a new book")
    print("2. Remove a book")
    print("3. Search for a book")
    print("4. List all books")
    print("5. Borrow a book")
    print("6. Return a book")
    print("7. Reserve a book")
    print("8. List borrowed books")
    print("9. Save library data")
    print("10. Load library data")
    print("11. Generate report")
    print("12. Add user")
    print("13. Remove user")
    print("14. Exit")

def get_user_choice():
    try:
        choice = int(input("Enter your choice: "))
        return choice
    except ValueError:
        print("Please enter a valid integer.")
        return None

def log_operation(operation_type, username):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_entry = {
        "timestamp": timestamp,
        "operation": operation_type,
        "username": username
    }
    logs.append(log_entry)
    if len(logs) > 100:
        logs.pop(0)

def load_library_data():
    if os.path.exists("books.csv"):
        return pd.read_csv("books.csv")
    else:
        return pd.DataFrame(columns=["title", "author", "year_of_publication"])

def save_library_data(df):
    df.to_csv("books.csv", index=False)

def load_user_data():
    if os.path.exists("user.csv"):
        return pd.read_csv("user.csv")
    else:
        return pd.DataFrame(columns=["username", "password", "role"])

def save_user_data(df):
    df.to_csv("user.csv", index=False)

def load_borrowed_books():
    if os.path.exists("borrowed_books.csv"):
        return pd.read_csv("borrowed_books.csv", parse_dates=["borrowed_date"])
    else:
        return pd.DataFrame(columns=["title", "borrower", "borrowed_date"])

def save_borrowed_books(df):
    df.to_csv("borrowed_books.csv", index=False)

def handle_add_book():
    library_books_df = load_library_data()

    name = input("Enter username: ")
    passwd = input("Enter password: ")

    if name == admin_username and passwd == admin_pass:
        title = input("Enter the title of the book: ")
        author = input("Enter the author's name: ")
        yop = int(input("Enter the year of publication: "))
        if yop < 0 or yop > datetime.datetime.now().year:
            print("Invalid year of publication.\n")
            return

        new_book_df = pd.DataFrame({
            "title": [title],
            "author": [author],
            "year_of_publication": [yop]
        })

        library_books_df = pd.concat([library_books_df, new_book_df], ignore_index=True)
        save_library_data(library_books_df)
        print(f"\nBook '{title}' added successfully.\n")
        log_operation("Add Book", name)
    else:
        print("Invalid username or password.\n")

def handle_remove_book():
    library_books_df = load_library_data()  

    name = input("Enter username: ")
    passwd = input("Enter password: ")

    if name == admin_username and passwd == admin_pass:
        title = input("Enter the title of the book to remove: ")
        if not title.strip():
            print("Error: Title cannot be empty.")
            return
        if title in library_books_df['title'].values:
            library_books_df = library_books_df[library_books_df['title'] != title]
            save_library_data(library_books_df)
            print(f"\nBook '{title}' removed successfully.")
            log_operation("Remove Book", name)
        else:
            print(f"\nNo book found with the title '{title}'.")
    else:
        print("Invalid username or password.\n")

def handle_search_book():
    library_books_df = load_library_data() 

    def by_title():
        search_title = input("Enter the title: ")
        results = library_books_df[library_books_df['title'].str.lower() == search_title.lower()]

        if not results.empty:
            for _, book in results.iterrows():
                print(f"\nBook found:\nTitle: {book['title']}\nAuthor: {book['author']}\nYear of Publication: {book['year_of_publication']}")
        else:
            print(f"\nNo book found with the title '{search_title}'")

    def by_author():
        search_author = input("Enter the author's name: ")
        results = library_books_df[library_books_df['author'].str.lower() == search_author.lower()]

        if not results.empty:
            for _, book in results.iterrows():
                print(f"\nBook found:\nTitle: {book['title']}\nAuthor: {book['author']}\nYear of Publication: {book['year_of_publication']}")
        else:
            print(f"\nNo book found with the author '{search_author}'")

    def by_yop():
        search_yop = input("Enter the year of publication: ")
        results = library_books_df[library_books_df['year_of_publication'].astype(str) == search_yop]

        if not results.empty:
            for _, book in results.iterrows():
                print(f"\nBook found:\nTitle: {book['title']}\nAuthor: {book['author']}\nYear of Publication: {book['year_of_publication']}")
        else:
            print(f"\nNo book found with the year of publication '{search_yop}'")

    def display_menu():
        print("\nSearch by:")
        print("1. Title")
        print("2. Author")
        print("3. Year of Publication")
        print("4. Exit")

    while True:
        display_menu()
        choice = get_user_choice()
        if choice == 1:
            by_title()
        elif choice == 2:
            by_author()
        elif choice == 3:
            by_yop()
        elif choice == 4:
            break 
        else:
            display_invalid_choice_message()

def handle_list_books():
    library_books_df = load_library_data()

    if not library_books_df.empty:
        print("\nBooks available in the library:")
        for idx, book in library_books_df.iterrows(): 
            print(f"{idx+1}. Title: {book['title']}\n   Author: {book['author']}\n   Year of Publication: {book['year_of_publication']}\n")
    else:
        print("\nNo books available in the library.\n")

def handle_borrow_book():
    title = input("Enter the title of the book you want to borrow: ").strip()
    if not title:
        print("Invalid input. The book title cannot be an empty string.\n")
        return
    library_books_df = load_library_data()
    borrowed_books_df = load_borrowed_books()

    if title.lower() in library_books_df['title'].str.lower().values:
        if title.lower() in borrowed_books_df['title'].str.lower().values:
            print(f"Sorry, the book '{title}' has already been borrowed.\n")
            return

        borrower = input("Enter your username: ").strip()
        users_df = load_user_data()
        
        if borrower not in users_df['username'].values:
            print(f"User '{borrower}' is not registered. Please register first.\n")
            return

        new_borrowed_book_df = pd.DataFrame({
            "title": [title], 
            "borrower": [borrower], 
            "borrowed_date": [datetime.datetime.now()]
        })
        if not new_borrowed_book_df.empty:
            borrowed_books_df = pd.concat([borrowed_books_df, new_borrowed_book_df], ignore_index=True)
            save_borrowed_books(borrowed_books_df)
        
        print(f"You have successfully borrowed '{title}'.\n")
        log_operation("Borrow Book", borrower)
    else:
        print(f"Sorry, the book '{title}' is not available in the library.\n")

def handle_reserve_book():
    title = input("Enter the title of the book you want to reserve: ").strip()



def handle_return_book():
    title = input("Enter the title of the book to return: ").strip()

    if not title:
        print("Invalid input. The book title cannot be an empty string.\n")
        return
    
    borrowed_books_df = load_borrowed_books()
    found = borrowed_books_df[borrowed_books_df['title'].str.lower() == title.lower()]

    if not found.empty:
        borrowed_books_df = borrowed_books_df[borrowed_books_df['title'].str.lower() != title.lower()]
        save_borrowed_books(borrowed_books_df)
        print(f"Book '{title}' returned successfully.\n")
    else:
        print(f"No record found for the borrowed book titled '{title}'.\n")

def handle_list_borrowed_books():
    borrowed_books_df = load_borrowed_books()
    
    if not borrowed_books_df.empty:
        print("\nCurrently borrowed books:")
        for _, book in borrowed_books_df.iterrows():
            print(f"Title: {book['title']}, Borrower: {book['borrower']}, Borrowed Date: {book['borrowed_date']}")
    else:
        print("\nNo borrowed books found.\n")

def handle_add_user():
    name = input("Enter username: ")
    passwd = input("Enter password: ")

    if name == admin_username and passwd == admin_pass:
        users_df = load_user_data()
        role = input("Enter role (user/librarian): ").strip().lower()

        if role not in ["user", "librarian"]:
            print("Invalid role. Choose either 'user' or 'librarian'.")
            return
        
        u_name = input(f"Enter new {role}'s username: ")
        u_pass = input(f"Create a password for {role}: ")
        
        if users_df[users_df['username'] == u_name].empty:
            new_user_df = pd.DataFrame({
                "username": [u_name],
                "password": [u_pass],
                "role": [role]
            })
            users_df = pd.concat([users_df, new_user_df], ignore_index=True)
            save_user_data(users_df) 
            print(f"\n{role.capitalize()} '{u_name}' added successfully!\n")
        else:
            print(f"\nError: Username '{u_name}' already exists. Try a different username.\n")
    else:
        print("Invalid admin credentials.\n")

def handle_remove_user():
    users_df = load_user_data()  

    name = input("Enter username: ")
    passwd = input("Enter password: ")

    if name == admin_username and passwd == admin_pass:
        u_name = input("Enter the username to remove: ")
        if not u_name.strip():
            print("Error: username cannot be empty.")
            return
        if u_name in users_df['username'].values:
            users_df = users_df[users_df['username'] != u_name]
            save_user_data(users_df)
            print(f"\n'{u_name}' removed successfully.")
        else:
            print(f"\nNo user found with username: '{u_name}'.")
    else:
        print("Invalid admin credentials.\n")

def handle_generate_report():
    library_books_df = load_library_data()
    borrowed_books_count = len(borrowed_books)
    print("\n--- Library Report ---")
    print(f"Total books in library: {len(library_books_df)}")
    print(f"Total borrowed books: {borrowed_books_count}")
    print("\n--- Log Entries ---")
    if logs:
        for log in logs:
            print(f"{log['timestamp']} - {log['operation']} by {log['username']}")
    else:
        print("No log entries available.")

def handle_save_library_data():
    pass

def handle_load_library_data():
    pass

def display_invalid_choice_message():
    print("Invalid choice. Please try again.")

def main():
    while True:
        display_main_menu()
        choice = get_user_choice()
        if choice is None:
            continue
        
        if choice == 1:
            handle_add_book()
        elif choice == 2:
            handle_remove_book()
        elif choice == 3:
            handle_search_book()
        elif choice == 4:
            handle_list_books()
        elif choice == 5:
            handle_borrow_book()
        elif choice == 6:
            handle_return_book()
        elif choice == 7:
            handle_reserve_book()  
        elif choice == 8:
            handle_list_borrowed_books()
        elif choice == 9:
            handle_save_library_data() 
        elif choice == 10:
            handle_load_library_data()  
        elif choice == 11:
            handle_generate_report()  
        elif choice == 12:
            handle_add_user()
        elif choice == 13:
            handle_remove_user()
        elif choice == 14:
            print("Exiting the system. Goodbye!")
            break
        else:
            display_invalid_choice_message()


In [20]:
main()

Library Management System
1. Add a new book
2. Remove a book
3. Search for a book
4. List all books
5. Borrow a book
6. Return a book
7. Reserve a book
8. List borrowed books
9. Save library data
10. Load library data
11. Generate report
12. Add user
13. Remove user
14. Exit

--- Library Report ---
Total books in library: 5
Total borrowed books: 0

--- Log Entries ---
No log entries available.
Library Management System
1. Add a new book
2. Remove a book
3. Search for a book
4. List all books
5. Borrow a book
6. Return a book
7. Reserve a book
8. List borrowed books
9. Save library data
10. Load library data
11. Generate report
12. Add user
13. Remove user
14. Exit

Books available in the library:
1. Title: Book Title 1
   Author: Author 1
   Year of Publication: 2020

2. Title: Book Title 2
   Author: Author 2
   Year of Publication: 2019

3. Title: The books
   Author: abc
   Year of Publication: 2022

4. Title: ABC
   Author: Anitha
   Year of Publication: 2021

5. Title: Happiness
 